In [1]:
%matplotlib inline

import cosima_cookbook as cc
import matplotlib.pyplot as plt
import numpy as np
import netCDF4 as nc
import xarray as xr
import cmocean as cm
from collections import OrderedDict
import cftime
import glob
import copy

import logging
logging.captureWarnings(True)
logging.getLogger('py.warnings').setLevel(logging.ERROR)

from dask.distributed import Client

In [2]:
#client = Client(n_workers=4)
# >> dask-scheduler
# >> dask-worker tcp://10.0.64.9:8786 --memory-limit 4e9 --nprocs 6 --nthreads 1 --local-directory /local/g40/amh157
#client = Client('tcp://10.0.64.9:8786', local_dir='/local/g40/amh157')
client = Client(n_workers=8)
client

Client Scheduler: tcp://127.0.0.1:34465 Dashboard: /proxy/8787/status,Cluster Workers: 8 Cores: 8 Memory: 68.72 GB


In [5]:
#db = '/scratch/e14/rmh561/access-om2/archive/databases/cc_database_omip'
session = cc.database.create_session()

In [87]:
# define locations, time-ranges, variables, etc.:
expt = '025deg_jra55_iaf_omip2_cycle6'
dates = ['1995-01-01','2050-12-31']
variables = ['temp','salt','net_sfc_heating']
locations = {'10W0N': (-10.,0), '23W0N': (-23.,0.), '140W0N': (-140.,0.), '110W0N': (-110.,0.)}

# Output location:
base = '/g/data/e14/rmh561/'

## Extract data at one location

In [88]:
locs = locations.keys()#['140W0N']

sw_ocean = cc.querying.getvar(expt,'sw_ocean', session, start_time=dates[0], end_time=dates[1])
#st_ocean = cc.querying.getvar(expt,'st_ocean', session, start_time=dates[0], end_time=dates[1])

for loc in locs:
    lon = locations[loc][0]
    lat = locations[loc][1]
    
    JqC = cc.querying.getvar(expt, 'temp_vdiffuse_diff_cbt', session,
                            start_time=dates[0],
                            end_time=dates[1]).sel(xt_ocean=lon,method='nearest').sel(yt_ocean=lat,method='nearest')
    JqC.values = JqC.cumsum('st_ocean').values
    JqC = JqC.rename({'st_ocean':'sw_ocean'}).assign_coords({'sw_ocean': sw_ocean})
    
    swC = cc.querying.getvar(expt, 'sw_heat', session,
                            start_time=dates[0],
                            end_time=dates[1]).sel(xt_ocean=lon,method='nearest').sel(yt_ocean=lat,method='nearest')
    swC.values = swC.cumsum('st_ocean').values
    swC = swC.rename({'st_ocean':'sw_ocean'}).assign_coords({'sw_ocean': sw_ocean})
    
    ovars = {}
    for var in variables:
        ovars[var] = cc.querying.getvar(expt, var, session,
                            start_time=dates[0],
                            end_time=dates[1]).sel(xt_ocean=lon,method='nearest').sel(yt_ocean=lat,method='nearest')
    
    out = xr.merge([JqC,swC,ovars])
    out.to_netcdf(base + loc + '.nc')

In [86]:
out.temp.time.isel(time=-1)

<xarray.DataArray 'time' ()>
array(cftime.DatetimeGregorian(2018, 12, 14, 12, 0, 0, 0), dtype=object)
Coordinates:
    xt_ocean  float64 -109.9
    yt_ocean  float64 0.125
    time      object 2018-12-14 12:00:00
Attributes:
    long_name:       time
    cartesian_axis:  T
    calendar_type:   GREGORIAN
    bounds:          time_bounds